3/5/2022: We ran 1x1 cadences and 174x8 would take forever. Let's hack the fits files to make them appear to be a 174x8. 

To do:
-  change NEXP in header (implemented, imp)
-  change FCADENCE in header (imp)
-  change DESMODE in header (imp)
- CHANGE TO REPEAT DESMODE AND SKIES AND STANDARDS LIKE EXAMPLE (1392 TIMES) (imp)
- **change cadence data column to dark_174x8**
-  **check structure of a 174x8**
- **change hdu 2 data to match structure (I think hdu1 is fine now?)**

In [1]:
#I need the header to look at NEXP
from astropy.io import fits
import glob
import numpy as np
from astropy.table import Table


example = "RM_survey2.fits" #a 174x8
survey_files = glob.glob('*MAX.fits')
print(survey_files)
epoch = 174
exposures = 1392

['COSMOS_surveyMAX.fits', 'RM_surveyMAX.fits', 'XMMLSS_surveyMAX.fits']


In [2]:
for i in survey_files:
    print(i)
    head = fits.open(i)[0].header
    print(head['RACEN'])
    print(head['DECCEN'])

COSMOS_surveyMAX.fits
150.01675
2.18325
RM_surveyMAX.fits
213.7041666
53.075
XMMLSS_surveyMAX.fits
35.77533
-5.0165


## Edit Header, Save

In [3]:
# #Open the example #NECESSARY, DO NOT DELETE
examp = fits.open(example)

In [ ]:
# #THIS ASSUMES THAT ALL OF THEM HAD 50 SKIES and 70 STANDARDS!!!!!!
# #CHECK THAT OR ELSE YOU COULD DESTROY SDSS MEG

# for i in range(len(survey_files)):
#     surv_file = fits.open(survey_files[i])

#     surv_file[0].header['DESMODE'] = examp[0].header['DESMODE']
#     surv_file[0].header['ACNUM0'] = examp[0].header['ACNUM0']
#     surv_file[0].header['ACNUM1'] = examp[0].header['ACNUM1']
#     surv_file[0].header['ACNUM2'] = examp[0].header['ACNUM2']
#     surv_file[0].header['ACNUM3'] = examp[0].header['ACNUM3']
#     surv_file[0].header['FCADENCE'] = 'dark_174x8'
#     surv_file[0].header['NEXP'] = 1392
#     print(surv_file[0].header)
#     #surv_file.writeto('{}_surveyMAXHACKED.fits'.format(survey_files[i].split("_")[0]))

In [ ]:
# data1 = Table.read(survey_filesHACKED[i], format='fits')
# data1.

In [36]:
#columns to change
#key= col name
#item = shape (#,)

#THIS ASSUMES THAT ALL OF THEM HAD 50 SKIES and 70 STANDARDS!!!!!!
#CHECK THAT OR ELSE YOU COULD DESTROY SDSS MEG

change_cols = {'allowed': epoch, 'mags_allowed': epoch,'bright_allowed': epoch, 
           "robotID":exposures, "holeID":exposures,'equivRobotID':exposures, 
           'target_skybrightness':exposures, 'field_skybrightness':exposures, 
           'scienceRobotID':exposures}

for i in survey_files:
    print(i)
    surv_file = fits.open(i)
    print(surv_file[0].header['RACEN'])
    print(surv_file[0].header['DECCEN'])
    surv_file[0].header['DESMODE'] = examp[0].header['DESMODE']
    surv_file[0].header['ACNUM0'] = examp[0].header['ACNUM0']
    surv_file[0].header['ACNUM1'] = examp[0].header['ACNUM1']
    surv_file[0].header['ACNUM2'] = examp[0].header['ACNUM2']
    surv_file[0].header['ACNUM3'] = examp[0].header['ACNUM3']
    surv_file[0].header['FCADENCE'] = 'dark_174x8'
    surv_file[0].header['NEXP'] = 1392
    
    
    data1 = Table.read(i, hdu=1, format='fits')
    data2 = Table.read(i, hdu=2, format='fits')

    data1['tag'] = np.full(len(data1['catalogid']), '0.3.5') #same regardless of field
    data1['plan'] = np.full(len(data1['catalogid']), '0.5.4') #same
    data1['cadence'] = np.full(len(data1['catalogid']), 'dark_174x8') #same

    #mapper stuff only correlates to the science targets
    data1['mapper'] = np.full(len(data1['catalogid']), "   ") #same
    data1['mapper'][data1['category'] == 'science'] = 'BHM'

    data2['nexps'][data2['nexps'] == 1] = exposures
    data2['nepochs'][data2['nepochs'] == 1] = epoch
    
    for key in change_cols.keys():
        new_column = []
        for j in range(len(data2[key])):
            new_entry = np.full(change_cols[key], data2[key][j])
            new_column.append(new_entry)
        data2[key] = new_column
    
    hdu_list = fits.HDUList([fits.PrimaryHDU(header =surv_file[0].header),
    fits.table_to_hdu(data1),
    fits.table_to_hdu(data2)])
    
    hdu_list.writeto('{}_surveyMAXHACKED.fits'.format(i.split("_")[0]), overwrite =True)
    
    print("SAVING HACKED FILE.")


COSMOS_surveyMAX.fits
150.01675
2.18325
SAVING HACKED FILE.
RM_surveyMAX.fits
213.7041666
53.075
SAVING HACKED FILE.
XMMLSS_surveyMAX.fits
35.77533
-5.0165
SAVING HACKED FILE.


In [39]:
survey_filesHACKED = glob.glob('*MAXHACKED.fits')
print(survey_filesHACKED)
i = 0
print(survey_filesHACKED[i])
head = fits.open(survey_filesHACKED[i])[0].header
data1 = Table.read(survey_filesHACKED[i], hdu=1, format='fits')
data2 = Table.read(survey_filesHACKED[i], hdu=2, format='fits')
print(head['RACEN'])
print(head['DECCEN'])
data2

['COSMOS_surveyMAXHACKED.fits', 'XMMLSS_surveyMAXHACKED.fits', 'RM_surveyMAXHACKED.fits']
COSMOS_surveyMAXHACKED.fits
150.01675
2.18325


assigned,satisfied,science_satisfied,extra,nexps,nepochs,allowed [174],mags_allowed [174],bright_allowed [174],robotID [1392],holeID [1392],equivRobotID [1392],scienceRobotID [1392],target_skybrightness [1392],field_skybrightness [1392],fiberType,rsflags,expflag
int32,int32,int32,int32,int32,int32,bool,bool,bool,int32,bytes28,int32,int32,float32,float32,bytes10,int32,int32
0,0,0,0,0,0,True .. True,True .. True,True .. True,-1 .. -1,..,-1 .. -1,-1 .. -1,-1.0 .. -1.0,0.35 .. 0.35,BOSS,0,0
0,0,0,0,0,0,True .. True,True .. True,True .. True,-1 .. -1,..,-1 .. -1,-1 .. -1,-1.0 .. -1.0,0.35 .. 0.35,BOSS,0,0
1,1,0,0,1392,174,True .. True,True .. True,True .. True,1317 .. 1317,R-12C9 .. R-12C9,1317 .. 1317,-1 .. -1,-1.0 .. -1.0,0.35 .. 0.35,BOSS,0,2
0,0,0,0,0,0,True .. True,True .. True,True .. True,-1 .. -1,..,-1 .. -1,-1 .. -1,-1.0 .. -1.0,0.35 .. 0.35,BOSS,0,0
1,1,0,0,1392,174,True .. True,True .. True,True .. True,624 .. 624,R-5C1 .. R-5C1,624 .. 624,-1 .. -1,-1.0 .. -1.0,0.35 .. 0.35,BOSS,0,2
1,1,0,0,1392,174,True .. True,True .. True,True .. True,663 .. 663,R0C1 .. R0C1,663 .. 663,-1 .. -1,-1.0 .. -1.0,0.35 .. 0.35,BOSS,0,2
1,1,0,0,1392,174,True .. True,True .. True,True .. True,550 .. 550,R+1C1 .. R+1C1,550 .. 550,-1 .. -1,-1.0 .. -1.0,0.35 .. 0.35,BOSS,0,2
1,1,0,0,1392,174,True .. True,True .. True,True .. True,1381 .. 1381,R-7C10 .. R-7C10,1381 .. 1381,-1 .. -1,-1.0 .. -1.0,0.35 .. 0.35,BOSS,0,2
1,1,0,0,1392,174,True .. True,True .. True,True .. True,1054 .. 1054,R-2C7 .. R-2C7,1054 .. 1054,-1 .. -1,-1.0 .. -1.0,0.35 .. 0.35,BOSS,0,2


In [17]:

# hdu_list = fits.HDUList([
#     fits.PrimaryHDU(header =fits.open(survey_filesHACKED[i])[0].header),
#     fits.table_to_hdu(data1),
#     fits.table_to_hdu(data2), 
# ])
# hdu_list.writeto('tables.fits', overwrite = True)

assigned,satisfied,science_satisfied,extra,nexps,nepochs,allowed,mags_allowed,bright_allowed,robotID,holeID,equivRobotID,scienceRobotID,target_skybrightness,field_skybrightness,fiberType,rsflags,expflag
int32,int32,int32,int32,int32,int32,bool,bool,bool,int32,bytes15,int32,int32,float32,float32,bytes10,int32,int32
0,0,0,0,0,0,True,True,True,-1,,-1,-1,-1.0,0.35,BOSS,0,0
0,0,0,0,0,0,True,True,True,-1,,-1,-1,-1.0,0.35,BOSS,0,0
1,1,0,0,1392,174,True,True,True,1317,R-12C9,1317,-1,-1.0,0.35,BOSS,0,2
0,0,0,0,0,0,True,True,True,-1,,-1,-1,-1.0,0.35,BOSS,0,0
1,1,0,0,1392,174,True,True,True,624,R-5C1,624,-1,-1.0,0.35,BOSS,0,2
1,1,0,0,1392,174,True,True,True,663,R0C1,663,-1,-1.0,0.35,BOSS,0,2
1,1,0,0,1392,174,True,True,True,550,R+1C1,550,-1,-1.0,0.35,BOSS,0,2
1,1,0,0,1392,174,True,True,True,1381,R-7C10,1381,-1,-1.0,0.35,BOSS,0,2
1,1,0,0,1392,174,True,True,True,1054,R-2C7,1054,-1,-1.0,0.35,BOSS,0,2


In [ ]:
#only issue?
head['BITPIX'] = 16

In [ ]:
surv_file = fits.open(survey_files[0])
len(surv_file[2].data['allowed'])

In [ ]:
# survey_filesHACKED = glob.glob('*MAXHACKED.fits')
# # survey_filesHACKED
# #columns to change
# #key= col name
# #item = shape (#,)

# change_cols = {'allowed': epoch, 'mags_allowed': epoch,'bright_allowed': epoch, 
#                "robotID":exposures, "holeID":exposures,'equivRobotID':exposures, 
#                'target_skybrightness':exposures, 'field_skybrightness':exposures, 
#                'scienceRobotID':exposures}

# for key in change_cols.keys():
#     new_column = []
#     for i in range(len(data2[key])):
#         new_entry = np.full(change_cols[key], data2[key][i])
#         new_column.append(new_entry)
#     data2[key] = new_column